In [1]:
import pandas as pd
import pickle

input_file = 'articles_preprocessed_1mio.pkl'
dfs = []

with open(input_file, "rb") as file:
    while True:
        try:
            chunk = pickle.load(file)
            dfs.append(chunk)
        except EOFError:
            break
df = pd.concat(dfs, ignore_index=True)
df.shape

(686801, 2)

In [2]:
print(type(df.loc[0, "content"]))#Debug

<class 'list'>


In [3]:
from collections import Counter
def get_vocab(article_series, n):
    """Takes a Pandas series of lists of words and returns a dictionary
        of the n most common words and their frequencies.
    """
    all_words = [word 
                for article in article_series 
                if isinstance(article, (list,str))
                for word in (article if isinstance(article, list) else article.split())
    ]
    word_counts = Counter(all_words)
    return dict(word_counts.most_common(n))

In [4]:
vocabulary = get_vocab(df['content'], 10000)

In [5]:
import numpy as np
import scipy.sparse as sp

def one_hot_encoding_2_sparse(article_series, vocabulary, dtype = np.int8):
    """
    input: article_series: pd.Series (list of words per article)
           vocabulary: dict (words as keys)
    output: pd.DataFrame (one-hot encoded word presence matrix)
    """
    vocab_list = list(vocabulary.keys())  # Ensure consistent ordering
    vocab_index = {word: i for i, word in enumerate(vocab_list)}  # Word-to-index mapping
    
    row_indices = []
    col_indices = []
    
    for i, article in enumerate(article_series):
        if isinstance(article, list):
            for word in article:
                if word in vocab_index:
                    row_indices.append(i)
                    col_indices.append(vocab_index[word])

    # Create a sparse matrix
    encoded_matrix = sp.csr_matrix(
        (np.ones(len(row_indices), dtype=dtype), (row_indices, col_indices)),
        shape=(len(article_series), len(vocab_list))
    )
    
    return encoded_matrix

In [6]:
articles_encoded = one_hot_encoding_2_sparse(df['content'],vocabulary)

In [7]:
import pickle

output_file = 'articles_one-hot-encoded_1mio_sparsem.pkl'

with open(output_file, "wb") as f:
    pickle.dump(articles_encoded,f)